In [1]:
import numpy as np

In [ ]:
def norma(x, p=2):
    x = np.ravel(x)
    return np.sum(np.abs(x)**p)**(1/p)

In [ ]:
def multMat(A, x):
    A = np.array(A)
    x = np.array(x)
    res = np.zeros(A.shape[0])
    for i in range(A.shape[0]):
        tot = 0
        for k in range(A.shape[1]):
            tot += A[i, k] * x[k]
        res[i] = tot
    return res

def prodMat(A, B):
    if A.shape[1] != B.shape[0]:
        print("No se puede hacer el producto matricial")
        return
    res = np.zeros((A.shape[0], B.shape[1]))

    for i in range(res.shape[0]):
        for j in range(res.shape[1]):
            tot = 0
            for k in range(A.shape[1]):
                tot += A[i, k] * B[k, j]
            res[i, j] = tot

    return res

def prodPunto(v, w):
    return np.sum(v*w)

In [6]:
def aplicTrans(A, v):
    """
    Asume que el vector es columna
    """

    if A.shape[1] != v.shape[0]:
        print("No se puede aplicar la matriz al vector")
        return
    res = np.zeros((A.shape[0]))

    for i in range(res.shape[0]):
        tot = 0            
        for k in range(A.shape[1]):
            tot += A[i, k] * v[k]
        res[i] = tot

    return res

In [ ]:
def metpot2k(A, tol=1e-15, K=1000):
    N = A.shape[0]

    # Caso matriz nula → todos los autovalores son 0
    if norma(A, 2) < tol:
        v = np.random.rand(N)
        v = v / norma(v, 2)
        v = np.expand_dims(v, axis=0).T
        return v, 0.0, 0

    # vector inicial
    v = np.random.rand(N)
    v = v / norma(v, 2)

    v_monio = aplicTrans(A, v)
    v_monio = v_monio / norma(v_monio, 2)

    e = prodPunto(v_monio, v)
    k = 0
    
    while abs(e - 1) > tol and k < K:

        v = v_monio

        # doble aplicación
        w = aplicTrans(A, v)
        w = w / norma(w, 2)

        v_monio = aplicTrans(A, w)
        v_monio = v_monio / norma(v_monio, 2)

        e = prodPunto(v_monio, v)
        k += 1

    v_monio = np.expand_dims(v_monio, axis=0).T
    # lambd = prodMat(prodMat(v_monio.T, A), v_monio)[0][0]
    lambd = v_monio.T@A@v_monio[0][0]

    return v_monio, lambd, e-1


In [8]:
def diagRH(A, tol=1e-15, K=1000):

    if A.shape[0] != A.shape[1]:
        print("Matriz no cuadrada")
        return None

    N = A.shape[0]

    # 1x1 → trivial
    if N == 1:
        return np.eye(1), A.copy()

    # autovector dominante
    v1, lambda1, _ = metpot2k(A, tol, K)

    # e1
    e1 = np.zeros((N,1))
    e1[0][0] = 1.0

    # vector de Householder
    u = e1 - v1
    nu = norma(np.squeeze(u), 2)
    
    print('2 llego aca')

    # si v1 ≈ e1 → no hacer Householder (H=I)
    if nu < 1e-12:
        Hv1 = np.eye(N)
    else:
        factor = 2.0 / (nu * nu)
        # Hv1 = np.eye(N) - factor * prodMat(u, u.T)
        Hv1 = np.eye(N) - factor * u@u.T

    # Caso N=2 → ya diagonaliza
    if N == 2:
        S = Hv1
        # D = prodMat(prodMat(Hv1, A), Hv1.T)
        D = Hv1@A@Hv1.T
        return S, D

    # Construir B = H A H^T
    # B = prodMat(prodMat(Hv1, A), Hv1.T)
    B = Hv1@A@Hv1.T

    # Submatriz sin la primera fila/col
    Ahat = B[1:N, 1:N]

    # Recurrencia
    Shat, Dhat = diagRH(Ahat, tol, K)

    # Construyo D grande
    D = np.zeros((N, N))
    D[0][0] = lambda1
    for i in range(1, N):
        for j in range(1, N):
            D[i][j] = Dhat[i-1][j-1]

    # Extiendo Shat a N×N
    Hprod = np.zeros((N, N))
    Hprod[0][0] = 1.0
    for i in range(1, N):
        for j in range(1, N):
            Hprod[i][j] = Shat[i-1][j-1]

    # matriz de autovectores final
    # S = prodMat(Hv1, Hprod)
    S= Hv1@Hprod

    return S, D


In [9]:
import time

def diagRH(A, tol=1e-15, K=1000, nivel=1):
    """
    Diagonalización recursiva con Householder
    
    Args:
        A: matriz a diagonalizar
        tol: tolerancia
        K: iteraciones máximas
        nivel: nivel de recursión (para tracking)
    """
    # Marca el inicio de esta iteración
    inicio = time.time()
    
    if A.shape[0] != A.shape[1]:
        print("Matriz no cuadrada")
        return None
    
    N = A.shape[0]
    
    # 1x1 → trivial
    if N == 1:
        fin = time.time()
        tiempo_transcurrido = fin - inicio
        minutos = int(tiempo_transcurrido // 60)
        segundos = tiempo_transcurrido % 60
        print(f"Nivel {nivel} (N=1, caso base): {minutos}m {segundos:.3f}s")
        return np.eye(1), A.copy()
    
    # autovector dominante
    v1, lambda1, _ = metpot2k(A, tol, K)
    
    # e1
    e1 = np.zeros((N, 1))
    e1[0][0] = 1.0
    
    # vector de Householder
    u = e1 - v1
    nu = norma(np.squeeze(u), 2)
    
    print(f'Nivel {nivel}: procesando matriz {N}x{N}')
    
    # si v1 ≈ e1 → no hacer Householder (H=I)
    if nu < 1e-12:
        Hv1 = np.eye(N)
    else:
        factor = 2.0 / (nu * nu)
        Hv1 = np.eye(N) - factor * u @ u.T
    
    # Caso N=2 → ya diagonaliza
    if N == 2:
        S = Hv1
        D = Hv1 @ A @ Hv1.T
        
        fin = time.time()
        tiempo_transcurrido = fin - inicio
        minutos = int(tiempo_transcurrido // 60)
        segundos = tiempo_transcurrido % 60
        print(f"Nivel {nivel} (N=2, caso base): {minutos}m {segundos:.3f}s")
        
        return S, D
    
    # Construir B = H A H^T
    B = Hv1 @ A @ Hv1.T
    
    # Submatriz sin la primera fila/col
    Ahat = B[1:N, 1:N]
    
    # Recurrencia (pasa el nivel incrementado)
    Shat, Dhat = diagRH(Ahat, tol, K, nivel + 1)
    
    # Construyo D grande
    D = np.zeros((N, N))
    D[0][0] = lambda1
    for i in range(1, N):
        for j in range(1, N):
            D[i][j] = Dhat[i-1][j-1]
    
    # Extiendo Shat a N×N
    Hprod = np.zeros((N, N))
    Hprod[0][0] = 1.0
    for i in range(1, N):
        for j in range(1, N):
            Hprod[i][j] = Shat[i-1][j-1]
    
    # matriz de autovectores final
    S = Hv1 @ Hprod
    
    # Calcula y muestra el tiempo al terminar esta iteración
    fin = time.time()
    tiempo_transcurrido = fin - inicio
    minutos = int(tiempo_transcurrido // 60)
    segundos = tiempo_transcurrido % 60
    print(f"Nivel {nivel} (N={N}): {minutos}m {segundos:.3f}s")
    
    return S, D


# Ejemplo de uso (asumiendo que tienes metpot2k y norma definidas)
# A = np.random.rand(5, 5)
# A = (A + A.T) / 2  # hacer simétrica
# S, D = diagRH(A)

In [ ]:
#OTRA CON MEDICION DE TIEMPO
import time

def diagRH(A, tol=1e-15, K=1000, nivel=1):
    """
    Diagonalización recursiva con Householder
    
    Args:
        A: matriz a diagonalizar
        tol: tolerancia
        K: iteraciones máximas
        nivel: nivel de recursión (para tracking)
    """
    # Marca el inicio de esta iteración
    inicio = time.time()
    
    if A.shape[0] != A.shape[1]:
        print("Matriz no cuadrada")
        return None
    
    N = A.shape[0]
    
    # 1x1 → trivial
    if N == 1:
        fin = time.time()
        tiempo_transcurrido = fin - inicio
        minutos = int(tiempo_transcurrido // 60)
        segundos = tiempo_transcurrido % 60
        print(f"✓ Nivel {nivel} completado (matriz 1x1 - CASO TRIVIAL): {minutos}m {segundos:.3f}s")
        return np.eye(1), A.copy()
    
    # autovector dominante
    v1, lambda1, _ = metpot2k(A, tol, K)
    
    # e1
    e1 = np.zeros((N, 1))
    e1[0][0] = 1.0
    
    # vector de Householder
    u = e1 - v1
    nu = norma(np.squeeze(u), 2)
    
    # (Este print se movió al final)
    
    # si v1 ≈ e1 → no hacer Householder (H=I)
    if nu < 1e-12:
        Hv1 = np.eye(N)
    else:
        factor = 2.0 / (nu * nu)
        Hv1 = np.eye(N) - factor * u @ u.T
    
    # Caso N=2 → ya diagonaliza
    if N == 2:
        S = Hv1
        D = Hv1 @ A @ Hv1.T
        
        fin = time.time()
        tiempo_transcurrido = fin - inicio
        minutos = int(tiempo_transcurrido // 60)
        segundos = tiempo_transcurrido % 60
        print(f"✓ Nivel {nivel} completado (matriz 2x2 - CASO BASE): {minutos}m {segundos:.3f}s")
        
        return S, D
    
    # Construir B = H A H^T
    B = Hv1 @ A @ Hv1.T
    
    # Submatriz sin la primera fila/col
    Ahat = B[1:N, 1:N]
    
    # Recurrencia (pasa el nivel incrementado)
    Shat, Dhat = diagRH(Ahat, tol, K, nivel + 1)
    
    # Construyo D grande
    D = np.zeros((N, N))
    D[0][0] = lambda1
    for i in range(1, N):
        for j in range(1, N):
            D[i][j] = Dhat[i-1][j-1]
    
    # Extiendo Shat a N×N
    Hprod = np.zeros((N, N))
    Hprod[0][0] = 1.0
    for i in range(1, N):
        for j in range(1, N):
            Hprod[i][j] = Shat[i-1][j-1]
    
    # matriz de autovectores final
    S = Hv1 @ Hprod
    
    # Calcula y muestra el tiempo al terminar esta iteración
    fin = time.time()
    tiempo_transcurrido = fin - inicio
    minutos = int(tiempo_transcurrido // 60)
    segundos = tiempo_transcurrido % 60
    print(f"✓ Nivel {nivel} completado (matriz {N}x{N}): {minutos}m {segundos:.3f}s")
    
    return S, D


# Ejemplo de uso (asumiendo que tienes metpot2k y norma definidas)
# A = np.random.rand(5, 5)
# A = (A + A.T) / 2  # hacer simétrica
# S, D = diagRH(A)

In [10]:
def svd_reducida (A, k ='max' , tol=1e-15):
    
    n = A.shape[0]
    m = A.shape[1]
    
    if n < m: 
        # M = prodMat(A, A.T)
        M = A@A.T
    else:
        # M = prodMat(A.T, A)
        M = A.T@A
    S, D = diagRH(M, tol)
    
    aval = []
    avec = []
    for i in range (D.shape[0]):
        if D[i,i] > tol:
            aval.append(np.sqrt(D[i,i]))
            avec.append(S[:, i])
    
    # Aplica k si tiene limite
    if k != 'max':
        aval = aval[:k]
        avec = avec[:k]
    
    print('3 va por aqui')
    
    r = len(aval)
    S = np.array(aval)
    U = np.zeros((n, r))
    V = np.zeros((m,r))
    
    if n < m:
        for i in range(r):
            U[:, i] = avec[i] / norma(avec[i])
            # V_moño = multMat(A.T,U[:, i])
            V_moño = A.T@U[:, i]
            V[:, i] = V_moño / aval[i]
    else:
        for i in range(r):
            V[:, i] = avec[i] / norma(avec[i])
            # U_moño = multMat(A,V[:, i])
            U_moño = A@V[:, i]
            U[:,i] = U_moño / aval[i]
    
    
    return U, S, V

In [2]:
def fullyConnectedSVD(X,Y):
    U, S, V = svd_reducida(X)
    
    # #convertir S a S+
    # for i in range (S.shape):
    #     S[i] = 1/S[i]
    
    VS= np.zeros(V.shape)
    r = np.zeros[1]
    # en vez de pasar a S+ divido cada Vi por S[i] que es como multiplicar por 1/S[i] (inversa de sigma)
    for i in range(r):
        VS[:, i] = V[:, i]/S[i]
    
    X_ps = VS@U.T
    W = Y@X_ps
    
    return W

In [ ]:
def es_svd_valida(A, U, S, V, tol=1e-8):
    # 1. Chequear que U es ortonormal
    if not np.allclose(U.T @ U, np.eye(U.shape[1]), atol=tol):
        print("U no es ortonormal")
        # return False
    
    # 2. Chequear que V es ortonormal
    if not np.allclose(V.T @ V, np.eye(V.shape[1]), atol=tol):
        print("V no es ortonormal")
        # return False
    
    # 3. Chequear que S es diagonal
    if not np.allclose(S, np.diag(np.diag(S)), atol=tol):
        print("S no es diagonal")
        return False

    # 4. Chequear reconstrucción
    A_rec = U @ S @ V.T
    if not np.allclose(A, A_rec, atol=tol):
        print("A != U S V^T")
        return False
    
    return True

In [ ]:
# Después de calcular U y V
print("Ortogonalidad U:", np.max(np.abs(U.T @ U - np.eye(U.shape[1]))))
print("Ortogonalidad V:", np.max(np.abs(V.T @ V - np.eye(V.shape[1]))))
print("Reconstrucción:", np.max(np.abs(X_train - U @ np.diag(S) @ V.T)))

In [ ]:

from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def confusion_matrix_simple(W, X_val, Y_val):
    # 1. Predicciones
    Y_pred = Wsvd @ X_val
    y_pred = np.argmax(Y_pred, axis=0)
    y_true = np.argmax(Y_val, axis=0)

    # 2. Matriz de confusión
    M = np.zeros((2,2), dtype=int)
    for t,p in zip(y_true, y_pred):
        M[t,p] += 1

    # 3. Accuracy
    acc = np.trace(M) / np.sum(M)

    return M, acc

M, acc = confusion_matrix_simple(Wsvd, X_val, Y_val)
print("Matriz de confusión:\n", M)
print("Accuracy:", acc)


sns.heatmap(M, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.title('Matriz de Confusión')
plt.show()

In [1]:
def aplicTrans(A, v):
    """
    Asume que el vector es columna
    """

    if A.shape[1] != v.shape[0]:
        print("No se puede aplicar la matriz al vector")
        return
    res = np.zeros((A.shape[0]))
    for i in range (res.shape[0]):
        res[i] = prodPunto(A[i,:], v)

    # for i in range(res.shape[0]):
    #     tot = 0            
    #     for k in range(A.shape[1]):
    #         tot += A[i, k] * v[k]
    #     res[i] = tot

    return res

In [4]:
def prodPunto(v, w):
    """
    Asume que el vector es columna
    """

    return np.sum(v*w)

In [5]:
import numpy as np
A = np.array([[1,2,3],[1,2,3],[1,2,3]])
v = np.array([1,2,3])

print(aplicTrans(A,v))

[14. 14. 14.]


In [ ]:
import numpy as np

def prodMat(A, B):
    """
    Optimización del producto matricial A * B, eliminando un bucle anidado 
    mediante vectorización (sin usar np.dot o @).
    """
    if A.shape[1] != B.shape[0]:
        print("No se puede hacer el producto matricial")
        return
    
    A = A.astype(np.float64)
    B = B.astype(np.float64)
    res = np.zeros((A.shape[0], B.shape[1]), dtype=np.float64)
    
    # Pre-calculamos la transpuesta de B fuera del bucle para ahorrar tiempo
    B_T = B.T 

    # Bucle exterior (i) que itera sobre las filas de A (y C)
    for i in range(res.shape[0]):
        # 1. Multiplicación por Elementos con Broadcasting:
        # A[i, :] es la fila i de A, forma (N,).
        # B_T es la transpuesta de B, forma (P, N).
        # (N,) * (P, N) usa broadcasting, resultando en un array de forma (P, N)
        # donde cada fila de B_T se multiplicó por A[i, :].
        
        multiplicacion_broadcast = A[i, :] * B_T 
        
        # 2. Suma Vectorizada:
        # Sumamos a lo largo del eje 1 (a lo largo de la dimensión N) para
        # obtener el producto punto de A[i, :] con cada columna original de B.
        # El resultado es un vector de forma (P,) que es la fila i de la matriz C.
        res[i, :] = np.sum(multiplicacion_broadcast, axis=1)

    return res